### Notebook for constructiong POD-RBF NIROM approximation for a flow around a cylinder example

A collection of high-fidelity snapshots are generated that sufficiently capture the time-dynamics of the simulation. POD is adopted to define a reduced basis space for the high-fidelity snaphosts. RBF interpolation is adopted to approximate the evolution of the time dynamics in the reduced space spanned by the POD modes.  

OpenFOAM is used as the high-fidelity model for simulating flow around a cylinder governed by incompressible 2D Navier Stokes.  

In [ ]:
## Load modules
import numpy as np
import scipy
from importlib import reload

import os
import gc
from scipy import interpolate
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, ScalarFormatter, FormatStrFormatter

from matplotlib import animation
matplotlib.rc('animation', html='html5')
from IPython.display import display
import matplotlib.ticker as ticker
from matplotlib import rcParams
from matplotlib.offsetbox import AnchoredText

# Plot parameters
plt.rc('font', family='serif')
plt.rcParams.update({'font.size': 20,
                     'lines.linewidth': 2,
                     'axes.labelsize': 16, # fontsize for x and y labels (was 10)
                     'axes.titlesize': 20,
                     'xtick.labelsize': 16,
                     'ytick.labelsize': 16,
                     'legend.fontsize': 16,
                     'axes.linewidth': 2})

import itertools
colors = itertools.cycle(['r','g','b','m','y','c'])
markers = itertools.cycle(['p','d','o','^','s','x','D','H','v','*'])



base_dir = os.getcwd()
src_dir = os.path.join(base_dir,'../src/podrbf/')
work_dir = os.path.join(base_dir,'../notebooks/')
data_dir = os.path.join(base_dir,'../data/')
nirom_data_dir = os.path.join(base_dir,'../data/')
fig_dir = os.path.join(base_dir,'../figures/rbf/')



os.chdir(src_dir)
import main as prb
import plotting as pu
import rbf as rbf
import pod as pod
os.chdir(work_dir)



In [ ]:
## Load snapshot data
os.chdir(data_dir)
data = np.load('cylinder_Re100.0_Nn14605_Nt3001.npz')
mesh = np.load('OF_cylinder_mesh_Nn14605_Ne28624.npz')

print("Solution component keys are : " + str(data.keys()))
print("Mesh element keys are : " + str(mesh.keys()))
os.chdir(work_dir)

In [ ]:
## Prepare training snapshots
soln_names = ['p', 'v_x', 'v_y']
nodes = mesh['nodes'];  node_ind = mesh['node_ind']
triangles = mesh['elems']; elem_ind = mesh['elem_ind']

snap_start = 1250
T_end = 5.0   ### 5 seconds 

snap_data = {}
for key in soln_names:
    snap_data[key] = data[key][:,snap_start:]

times_offline = data['time'][snap_start:]
print('Loaded {0} snapshots of dimension {1} for h,u and v, spanning times [{2}, {3}]'.format(
                    snap_data[soln_names[0]].shape[1],snap_data[soln_names[0]].shape[0], 
                    times_offline[0], times_offline[-1]))


## number of steps to skip in selecting training snapshots for SVD basis
snap_incr=4
## Subsample snapshots for building POD basis
snap_end = np.count_nonzero(times_offline[times_offline <= T_end])
snap_train = {};
for key in soln_names:
    snap_train[key] = snap_data[key][:,0:snap_end+1:snap_incr]

times_train=times_offline[0:snap_end+1:snap_incr]
print('Using {0} training snapshots for time interval [{1},{2}]'.format(times_train.shape[0], 
                                        times_train[0], times_train[-1]))

del data
del mesh
gc.collect()

In [ ]:
## Instantiate the POD-RBF class
# os.chdir(src_dir)
# reload(prb)

# trunc_lvl = 0.9999995  ##WORKS BEST
trunc_lvl = 0.99
eps_train=0.1
PRB = prb.PODRBFBase(trunc = trunc_lvl)

In [ ]:
## Compute the POD basis for the space of snapshots
Phi, Sigma, Z_train = PRB.compute_pod_basis(snap_train, times_train)
dzdt_train = PRB._dzdt_train

# greedy = False
## Compute the RBF approximation of the time derivative of
## POD modal coefficients

A, rbf_centers, rbf_coeff = PRB.fit_rbf(Z_train, times_train, kernel='matern',eps=eps_train)


In [ ]:
## Define time domain for the online simulation
Tonline_end = 6.0
test_end = np.count_nonzero(times_offline[times_offline<=Tonline_end]) 

uniform_step=False
step_off_snaps=True
if uniform_step:
    dt = 0.01
    nt_online= int(2*(times_train[-1]-times_train[0])/dt)
    times_online = np.linspace(times_train[0],times_train[-1],nt_online+1)
    print('Trying to simulate interval [{0},{1}] with {2} steps and uniform dt = {3}'.format(times_online[0],
                                                                        times_online[-1],nt_online,dt))
elif step_off_snaps:
    #try to skip through the fine grid steps
    onl_incr = snap_incr-3
    times_online = times_offline[:test_end-1:onl_incr]
    nt_online = times_online.shape[0]
    print('Trying to simulate interval [{0},{1}] with {2} uniform skipped steps'.format(times_online[0],
                                                                    times_online[-1],nt_online))

In [ ]:
## Evaluate NIROM online at queried time points
# reload(prb)

## RBF NIROM solution

uh, zh = PRB.predict_time(times_online, use_greedy=False)

### Visualize domain and ROM results

In [ ]:
## --- Visualize the singular values
# os.chdir(src_dir)
# reload(pu)
pu.plot_sing_val(Sigma)
plt.ylabel('$\ln{|\sigma|}$',fontsize=16)
plt.xlabel('Modes', fontsize=16)

# os.chdir(fig_dir)
# plt.savefig('san_diego_sing_value.png',dpi=600,bbox_inches='tight')

In [ ]:
## --- Visualize l2-norm of time derivative of modal coefficients
dZdata_norm = {}
for ii,key in enumerate(soln_names):
    dZdata_norm[key] = np.linalg.norm(dzdt_train[key],axis=0);

fig = plt.figure(figsize=(12,3))
ky = soln_names[1]; ky1 = soln_names[2]; ky2 = soln_names[0]
t_indx = times_train[:-1]
plt.plot(t_indx[:],(dZdata_norm[ky][:]),'r-o', markersize=8,markevery=100,label='$u_x$',linewidth=2)
plt.plot(t_indx[:],(dZdata_norm[ky1][:]),'g-D', markersize=8,markevery=100,label='$u_y$',linewidth=2)
plt.plot(t_indx[:],(dZdata_norm[ky2][:]),'b-^', markersize=8,markevery=100,label='$p$',linewidth=2)

ax = plt.gca()
ax.xaxis.set_tick_params(labelsize=16)
ax.yaxis.set_tick_params(labelsize=16)
lg = plt.legend(fontsize=16,ncol=3,loc='upper right') 
plt.xlabel('Time (seconds)',fontsize=16)

# plt.savefig('greedy/sd_nirom_mode_norms.pdf', bbox_inches='tight')


In [ ]:
## --- Visualize the solution
# os.chdir(src_dir)
# reload(pu)

def set_label(key):
    if key == 'v_x':
        ky = 'u_x'
    elif key == 'v_y':
        ky = 'u_y'
    elif key == 'p':
        ky = 'p'
    return ky

key = 'v_x'; iplot = 250
iplot_true = np.argmin(np.abs(times_offline-times_online[iplot]))

fig = plt.figure(figsize=(16,6))

urom = uh[key][:,iplot]
utrue = snap_data[key][:,iplot_true]
print("comparing NIROM solution at t = {1:.2f} s and fine-grid solution at t = {0:.2f} s".format(
                                            times_offline[iplot_true], times_online[iplot]))

ax1 = plt.subplot(2,1,1); ax1.axis('off')
cf1, boundaries_interp = pu.viz_sol(urom,nodes,triangles)
ax1.set_title("NIROM solution\n $%1.5f<\mathbf{%s}<%1.5f$"%(np.amin(urom),set_label(key),
                                                             np.amax(urom)),fontsize=16)
cb1 = plt.colorbar(cf1, boundaries = boundaries_interp)

ax2 = plt.subplot(2,1,2); ax2.axis('off')
cf2, boundaries_true = pu.viz_sol(utrue,nodes,triangles)
ax2.set_title("HFM solution\n $%1.5f<\mathbf{%s}<%1.5f$"%(np.amin(utrue),set_label(key),
                                                             np.amax(utrue)),fontsize=16)
cb2 = plt.colorbar(cf2, boundaries = boundaries_true)

In [ ]:
## Visualize NIROM error
# os.chdir(src_dir)
# reload(pu)
print("comparing NIROM solution at t = {1:.2f} s and fine-grid solution at t = {0:.2f} s".format(
                                        times_offline[iplot_true], times_online[iplot]))

fig = plt.figure(figsize=(16,3))
ax1 = plt.subplot(1,1,1); ax1.axis('off')
cf = pu.viz_err(urom,utrue,nodes,triangles)
boundaries_err = [np.amin(urom-utrue), np.amax(urom-utrue)]
ax1.set_title("$%1.6f <$ $\mathbf{%s}$ Error $< %1.6f$\n Rel Error 2-norm : %2.6f"%(boundaries_err[0],set_label(key),
                boundaries_err[1],np.linalg.norm(urom-utrue)/np.linalg.norm(utrue)),fontsize=16)


In [ ]:
## Visually compare spatial RMS error evolution with time for each greedy solution
# os.chdir(src_dir)
# reload(pu)

fig = plt.figure(figsize=(16,4))

rms = rbf.err_comp(uh,snap_data,times_offline,times_online)
pu.plot_rms_err(rms,times_online,key, set_label(key)) 
lg=plt.legend(fontsize=20,ncol=2)
    
plt.xlabel('Time (seconds)',fontsize=16);
plt.title('RMS Error for $%s$'%(set_label(key)),fontsize=16);

In [ ]:
## Saving the ROM model
# os.chdir(nirom_data_dir)
# filename='podrbf_rom_cylinder'
# PRB.save_to_disk(filename,PRB)
# os.chdir(work_dir)

In [ ]:
## Save the NIROM solutions to disk

os.chdir(nirom_data_dir)
np.savez_compressed('cylinder_online_rbf',p=urbf['p'],v_x = urbf['v_x'], v_y = urbf['v_y'],
                                        zp=zrbf['p'],zv_x = zrbf['v_x'], zv_y = zrbf['v_y'],
                                        time=times_online)
os.chdir(work_dir)